In [69]:
import gurobipy as gp
from gurobipy import GRB

- 决策变量：$x_{ij}$: 生产第i种糖用Ingredientj的量
- 目标函数：价格-成本
- 约束：可供Ingredient、成分、订单要求

In [70]:
combos, profit = gp.multidict(
    {
        ("MilkCandy", "IngredientA"): 24 - 20,
        ("MilkCandy", "IngredientB"): 24 - 12,
        ("MilkCandy", "IngredientC"): 24 - 8,
        ("FruitCandy", "IngredientA"): 15 - 20,
        ("FruitCandy", "IngredientB"): 15 - 12,
        ("FruitCandy", "IngredientC"): 15 - 8,
    }
)

In [71]:
m = gp.Model("糖果生产分配问题")

In [72]:
x = m.addVars(combos, name="x")

In [73]:
m.setObjective(x.prod(profit), sense=GRB.MAXIMIZE)

In [74]:
m.addConstr(
    x["MilkCandy", "IngredientA"] >= 0.5 * x.sum("MilkCandy", "*"), name="成分要求1"
)
m.addConstr(
    x["MilkCandy", "IngredientB"] >= 0.25 * x.sum("MilkCandy", "*"), name="成分要求2"
)
m.addConstr(
    x["MilkCandy", "IngredientC"] <= 0.1 * x.sum("MilkCandy", "*"), name="成分要求3"
)
m.addConstr(
    x["FruitCandy", "IngredientA"] <= 0.4 * x.sum("FruitCandy", "*"), name="成分要求4"
)
m.addConstr(
    x["FruitCandy", "IngredientB"] <= 0.4 * x.sum("FruitCandy", "*"), name="成分要求5"
)
m.addConstr(
    x["FruitCandy", "IngredientC"] >= 0.15 * x.sum("FruitCandy", "*"),
    name="成分要求6",
)

<gurobi.Constr *Awaiting Model Update*>

In [75]:
m.addConstr(x.sum("MilkCandy", "*") >= 600, name="订单要求1")
m.addConstr(x.sum("FruitCandy", "*") >= 800, name="订单要求2")

<gurobi.Constr *Awaiting Model Update*>

In [76]:
m.addConstr(x.sum("*", "IngredientA") <= 500, name="原料限制1")
m.addConstr(x.sum("*", "IngredientB") <= 750, name="原料限制2")
m.addConstr(x.sum("*", "IngredientC") <= 625, name="原料限制3")

<gurobi.Constr *Awaiting Model Update*>

In [77]:
m.write("assign.lp")

In [78]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 11 rows, 6 columns and 30 nonzeros
Model fingerprint: 0x1ac81230
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [3e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 8e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 10 rows, 5 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3625000e+04   4.166667e+01   0.000000e+00      0s
       1    1.3125000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.312500000e+04


In [79]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 13125.0
Optimal solution:
x[MilkCandy,IngredientA] = 500.0
x[MilkCandy,IngredientB] = 500.0
x[MilkCandy,IngredientC] = 0.0
x[FruitCandy,IngredientA] = 0.0
x[FruitCandy,IngredientB] = 250.0
x[FruitCandy,IngredientC] = 625.0


这题我的答案和书里的答案不一样，但书里的答案显然错误。\
理由：原料A可供500kg，高级奶糖需要原料A含量超过50%，可知高级奶糖最多产1000kg，而书中答案是1175kg。